# Distribution Matching for Edge Weights in NetworkX Graphs

## Overview
In this notebook, we aim to identify which probability distribution most closely matches the edge weights of a given NetworkX graph. The distributions considered include:
- Uniform (-1, 1)
- Uniform (0, 1)
- Normal (0, 1)
- Lognormal (0, 1)
- Exponential $(\lambda = 0.2)$
- Standard Cauchy

## Approach
To determine the best matching distribution for the edge weights, we follow these steps:

### 1. Generate Reference Samples
We generate a large number of samples from each of the candidate distributions to serve as references for comparison.

### 2. Extract Edge Weights from the Graph
We read the NetworkX graph and extract the edge weights, which will be compared against the reference samples.

### 3. Define Goodness-of-Fit Function
We define a goodness-of-fit function using the Kolmogorov-Smirnov (K-S) test to measure how well the edge weights match each candidate distribution.

### 4. Parallel Processing for Scalability
To efficiently handle thousands of instances, we use parallel processing to compute the goodness-of-fit statistics for each set of edge weights.

### 5. Determine Best Matching Distribution
Finally, we determine the best matching distribution by identifying which candidate distribution has the lowest K-S statistic.

## Implementation
The implementation is divided into several sections:
1. **Import Libraries**: Import necessary libraries for data handling, statistical analysis, and parallel processing.
2. **Generate Reference Samples**: Generate and store reference samples for each candidate distribution.
3. **Extract Edge Weights**: Read the NetworkX graph and extract the edge weights.
4. **Goodness-of-Fit Function**: Define the K-S test function to compare distributions.
5. **Parallel Comparison**: Use parallel processing to compare the edge weights with each reference distribution.
6. **Determine Best Match**: Identify the best matching distribution based on the goodness-of-fit statistics.

By following this structured approach, we can efficiently and accurately determine the distribution that most closely matches the edge weights of the given graph.


In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import random
import os
from tqdm import tqdm
from scipy.stats import ks_2samp


experiment = "INFORMS-Revision-12-node-network"
# Load your graph (example using a random graph)
G = nx.erdos_renyi_graph(100, 0.1)  # Replace with your actual graph

# Add random weights to edges (for demonstration)
for (u, v, w) in G.edges(data=True):
    w['weight'] = np.random.uniform(-1, 1)  # Replace with actual weights if available

# Extract edge weights
edge_weights = np.array([w['weight'] for (u, v, w) in G.edges(data=True)])


In [23]:
n_samples = 100000

uniform_samples = [random.uniform(0, 1) for _ in range(n_samples)]
uniform_plus_samples = [random.uniform(-1, 1) for _ in range(n_samples)]
normal_samples = [random.normalvariate(0, 1) for _ in range(n_samples)]
exponential_samples = [random.expovariate(0.2) for _ in range(n_samples)]
lognormal_samples = [random.lognormvariate(0, 1) for _ in range(n_samples)]
cauchy_samples = [np.random.standard_cauchy() for _ in range(n_samples)]

distributions = {
    'uniform': uniform_plus_samples,
    'uniform_plus': uniform_samples,
    'normal': normal_samples,
    'log-normal': lognormal_samples,
    'exponential': exponential_samples,
    'cauchy': cauchy_samples
}

In [24]:

def goodness_of_fit(test_data, ref_data):
    return ks_2samp(test_data, ref_data).statistic

In [25]:

def test_goodness_of_fit_all():
    np.random.seed(42)  # For reproducibility
    random.seed(42)  # For reproducibility with random.expovariate

    # Generate sample data for each distribution
    test_data_uniform_minus1_1 = np.random.uniform(-1, 1, 1000)
    ref_data_uniform_minus1_1 = np.random.uniform(-1, 1, 1000)
    
    test_data_uniform_0_1 = np.random.uniform(0, 1, 1000)
    ref_data_uniform_0_1 = np.random.uniform(0, 1, 1000)
    
    test_data_normal_0_1 = np.random.normal(0, 1, 1000)
    ref_data_normal_0_1 = np.random.normal(0, 1, 1000)
    
    test_data_lognormal_0_1 = np.random.lognormal(0, 1, 1000)
    ref_data_lognormal_0_1 = np.random.lognormal(0, 1, 1000)
    
    test_data_exponential_0_2 = [random.expovariate(0.2) for _ in range(1000)]
    ref_data_exponential_0_2 = [random.expovariate(0.2) for _ in range(1000)]
    
    test_data_cauchy = np.random.standard_cauchy(1000)
    ref_data_cauchy = np.random.standard_cauchy(1000)
    
    # Test data drawn from one distribution and reference data from another
    test_data_mixed = np.random.uniform(0, 1, 1000)
    ref_data_mixed = np.random.normal(0, 1, 1000)

    # Compute K-S statistics using the goodness-of-fit function
    ks_stat_uniform_minus1_1 = goodness_of_fit(test_data_uniform_minus1_1, ref_data_uniform_minus1_1)
    ks_stat_uniform_0_1 = goodness_of_fit(test_data_uniform_0_1, ref_data_uniform_0_1)
    ks_stat_normal_0_1 = goodness_of_fit(test_data_normal_0_1, ref_data_normal_0_1)
    ks_stat_lognormal_0_1 = goodness_of_fit(test_data_lognormal_0_1, ref_data_lognormal_0_1)
    ks_stat_exponential_0_2 = goodness_of_fit(test_data_exponential_0_2, ref_data_exponential_0_2)
    ks_stat_cauchy = goodness_of_fit(test_data_cauchy, ref_data_cauchy)
    ks_stat_mixed = goodness_of_fit(test_data_mixed, ref_data_mixed)

    # Expected K-S statistics using scipy's ks_2samp directly
    expected_ks_stat_uniform_minus1_1 = ks_2samp(test_data_uniform_minus1_1, ref_data_uniform_minus1_1).statistic
    expected_ks_stat_uniform_0_1 = ks_2samp(test_data_uniform_0_1, ref_data_uniform_0_1).statistic
    expected_ks_stat_normal_0_1 = ks_2samp(test_data_normal_0_1, ref_data_normal_0_1).statistic
    expected_ks_stat_lognormal_0_1 = ks_2samp(test_data_lognormal_0_1, ref_data_lognormal_0_1).statistic
    expected_ks_stat_exponential_0_2 = ks_2samp(test_data_exponential_0_2, ref_data_exponential_0_2).statistic
    expected_ks_stat_cauchy = ks_2samp(test_data_cauchy, ref_data_cauchy).statistic
    expected_ks_stat_mixed = ks_2samp(test_data_mixed, ref_data_mixed).statistic

    # Assertions to check if the computed K-S statistics match the expected values
    assert np.isclose(ks_stat_uniform_minus1_1, expected_ks_stat_uniform_minus1_1), f"Uniform (-1, 1) test failed: {ks_stat_uniform_minus1_1} != {expected_ks_stat_uniform_minus1_1}"
    assert np.isclose(ks_stat_uniform_0_1, expected_ks_stat_uniform_0_1), f"Uniform (0, 1) test failed: {ks_stat_uniform_0_1} != {expected_ks_stat_uniform_0_1}"
    assert np.isclose(ks_stat_normal_0_1, expected_ks_stat_normal_0_1), f"Normal (0, 1) test failed: {ks_stat_normal_0_1} != {expected_ks_stat_normal_0_1}"
    assert np.isclose(ks_stat_lognormal_0_1, expected_ks_stat_lognormal_0_1), f"Lognormal (0, 1) test failed: {ks_stat_lognormal_0_1} != {expected_ks_stat_lognormal_0_1}"
    assert np.isclose(ks_stat_exponential_0_2, expected_ks_stat_exponential_0_2), f"Exponential (λ=0.2) test failed: {ks_stat_exponential_0_2} != {expected_ks_stat_exponential_0_2}"
    assert np.isclose(ks_stat_cauchy, expected_ks_stat_cauchy), f"Cauchy test failed: {ks_stat_cauchy} != {expected_ks_stat_cauchy}"
    assert np.isclose(ks_stat_mixed, expected_ks_stat_mixed), f"Mixed distribution test failed: {ks_stat_mixed} != {expected_ks_stat_mixed}"

    print("All tests passed!")

# Run the test
test_goodness_of_fit_all()

All tests passed!


# Read in instances


In [26]:


# Define the directory containing the graph files
directory_path = f'../{experiment}/best_graphs_12/'

# Initialize an empty list to store the graphs
graphs = []
filenames = os.listdir(directory_path)

# Iterate over all files in the directory
for filename in filenames:
    if filename.endswith('.graphml'):  # Modify this condition for other formats
        file_path = os.path.join(directory_path, filename)
        try:
            G = nx.read_graphml(file_path)  # Use appropriate read function for other formats
            graphs.append(G)
        except Exception as e:
            print(f'Failed to read {filename}: {e}')

# Print the number of graphs read
print(f'Total graphs read: {len(graphs)}')

Total graphs read: 284


In [27]:
def extract_edge_weights(graph):
    return [data.get('weight', 0) for _, _, data in graph.edges(data=True)]

edge_weights_list = [extract_edge_weights(graph) for graph in graphs]

In [28]:

# Parallel Processing for Goodness-of-Fit comparison
def compare_distribution(test_data, distributions):
    results = {}
    for dist_name, ref_data in distributions.items():
        results[dist_name] = goodness_of_fit(test_data, ref_data)
    return results


In [29]:
# Run the comparison with progress tracking
results = []
for edge_weights in tqdm(edge_weights_list, desc="Comparing distributions"):
    result = compare_distribution(edge_weights, distributions)
    results.append(result)

# Determine Best Match
def determine_best_match(results):
    best_matches = []
    for result in results:
        best_match = min(result, key=result.get)
        best_matches.append(best_match)
    return best_matches

best_matches = determine_best_match(results)

Comparing distributions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 284/284 [00:29<00:00,  9.79it/s]


In [30]:

d_instances = pd.read_csv(f"../{experiment}/final_evolved_instances_n_12_with_source.csv")

filenames = [file for file in filenames if '.graphml' in file]
# Create DataFrame to hold filenames and their corresponding best matches
new_weight_df = pd.DataFrame({'instance': filenames, 'weight_type': best_matches})



In [31]:
d_instances

,z_1,z_2,Source,Source_new,instance_class
0,1.390078,1.447221,target_point_1.3447058823529414_1.285294117647...,nearly_complete_bipartite_target_point_1.34470...,nearly_complete_bipartite
1,1.033745,1.863531,target_point_0.631578947368421_2.6789473684210...,nearly_complete_bipartite_target_point_0.63157...,nearly_complete_bipartite
2,-1.460238,0.859186,target_point_-1.6666666666666667_0.83333333333...,power_law_tree_target_point_-1.666666666666666...,power_law_tree
3,-0.153573,2.779136,target_point_0.8421052631578947_2.605263157894...,nearly_complete_bipartite_target_point_0.84210...,nearly_complete_bipartite
4,0.089295,2.514753,target_point_1.102941176470588_2.0658823529411...,nearly_complete_bipartite_target_point_1.10294...,nearly_complete_bipartite
...,...,...,...,...,...
279,-0.032185,1.630777,target_point_0.3441176470588235_2.145058823529...,nearly_complete_bipartite_target_point_0.34411...,nearly_complete_bipartite
280,-2.144231,-0.961832,target_point_-1.7999999999999998_-0.9071428571...,3_regular_graph_target_point_-1.79999999999999...,3_regular_graph
281,1.634686,0.857115,target_point_1.8205882352941174_0.794117647058...,uniform_random_target_point_1.8205882352941174...,uniform_random
282,-1.780164,0.523360,target_point_-2.0588235294117645_0.94117647058...,power_law_tree_target_point_-2.058823529411764...,power_law_tree


In [32]:
new_weight_df

,instance,weight_type
0,target_point_1.3447058823529414_1.285294117647...,uniform_plus
1,target_point_0.631578947368421_2.6789473684210...,log-normal
2,target_point_-1.6666666666666667_0.83333333333...,cauchy
3,target_point_0.8421052631578947_2.605263157894...,log-normal
4,target_point_1.102941176470588_2.0658823529411...,log-normal
...,...,...
279,target_point_0.3441176470588235_2.145058823529...,uniform_plus
280,target_point_-1.7999999999999998_-0.9071428571...,uniform
281,target_point_1.8205882352941174_0.794117647058...,log-normal
282,target_point_-2.0588235294117645_0.94117647058...,cauchy


In [33]:
merged_df = d_instances.merge(new_weight_df, left_on='Source', right_on='instance', how='left')

In [34]:
merged_df

,z_1,z_2,Source,Source_new,instance_class,instance,weight_type
0,1.390078,1.447221,target_point_1.3447058823529414_1.285294117647...,nearly_complete_bipartite_target_point_1.34470...,nearly_complete_bipartite,target_point_1.3447058823529414_1.285294117647...,uniform_plus
1,1.033745,1.863531,target_point_0.631578947368421_2.6789473684210...,nearly_complete_bipartite_target_point_0.63157...,nearly_complete_bipartite,target_point_0.631578947368421_2.6789473684210...,log-normal
2,-1.460238,0.859186,target_point_-1.6666666666666667_0.83333333333...,power_law_tree_target_point_-1.666666666666666...,power_law_tree,target_point_-1.6666666666666667_0.83333333333...,cauchy
3,-0.153573,2.779136,target_point_0.8421052631578947_2.605263157894...,nearly_complete_bipartite_target_point_0.84210...,nearly_complete_bipartite,target_point_0.8421052631578947_2.605263157894...,log-normal
4,0.089295,2.514753,target_point_1.102941176470588_2.0658823529411...,nearly_complete_bipartite_target_point_1.10294...,nearly_complete_bipartite,target_point_1.102941176470588_2.0658823529411...,log-normal
...,...,...,...,...,...,...,...
279,-0.032185,1.630777,target_point_0.3441176470588235_2.145058823529...,nearly_complete_bipartite_target_point_0.34411...,nearly_complete_bipartite,target_point_0.3441176470588235_2.145058823529...,uniform_plus
280,-2.144231,-0.961832,target_point_-1.7999999999999998_-0.9071428571...,3_regular_graph_target_point_-1.79999999999999...,3_regular_graph,target_point_-1.7999999999999998_-0.9071428571...,uniform
281,1.634686,0.857115,target_point_1.8205882352941174_0.794117647058...,uniform_random_target_point_1.8205882352941174...,uniform_random,target_point_1.8205882352941174_0.794117647058...,log-normal
282,-1.780164,0.523360,target_point_-2.0588235294117645_0.94117647058...,power_law_tree_target_point_-2.058823529411764...,power_law_tree,target_point_-2.0588235294117645_0.94117647058...,cauchy


In [35]:
# Read in each graph and add an attribute for the instance_class and weight_type
# Process each graph listed in the `instance` column of `merged_df`
for index, row in merged_df.iterrows():
    graph_file = row['instance']
    
    try:
        # Read the graph
        graph = nx.read_graphml(f"{directory_path}/{graph_file}")
        
        # Retrieve instance_class and weight_type
        instance_class = row['instance_class']
        weight_type = row['weight_type']
        
        # Add attributes to the graph
        graph.graph['instance_class'] = instance_class
        graph.graph['weight_type'] = weight_type
        
        # Save the graph with the new attributes
        nx.write_graphml(graph, f'../{experiment}/spartan-ready-instances/{graph_file}')  # Adjust the path as necessary
        
    except Exception as e:
        print(f"Error processing {graph_file}: {e}")


In [36]:
# Test that it worked
graph_file_to_check = f'../{experiment}/spartan-ready-instances/target_point_2.25_0.01_n_12_best_graph_gen_1276.graphml'

# Read the graph
graph = nx.read_graphml(graph_file_to_check)

# Retrieve the attributes
instance_class = graph.graph.get('instance_class', 'Attribute not found')
weight_type = graph.graph.get('weight_type', 'Attribute not found')

# Print the attributes
print(f"Graph: {graph_file_to_check}")
print(f"Instance Class: {instance_class}")
print(f"Weight Type: {weight_type}")


Graph: ../INFORMS-Revision-12-node-network/spartan-ready-instances/target_point_2.25_0.01_n_12_best_graph_gen_1276.graphml
Instance Class: geometric
Weight Type: cauchy
